# Spark Mini Exercises

In [1]:
# imports
import pandas as pd
import numpy as np

import pyspark
import pyspark.sql.functions as F

import multiprocessing

In [2]:
#spark = pyspark.sql.SparkSession.builder.getOrCreate()
#spark.range(5).show()

In [3]:
nprocs = multiprocessing.cpu_count()

spark = (pyspark.sql.SparkSession.builder
 .master('local')
 .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.16')
 .config('spark.driver.memory', '4G')
 .config('spark.driver.cores', nprocs)
 .config('spark.sql.shuffle.partitions', nprocs)
 .appName('MySparkApplication')
 .getOrCreate())

In [4]:
# Use the starter code above to create a pandas dataframe.
np.random.seed(13)

pandas_dataframe = pd.DataFrame({
    "n": np.random.randn(20),
    "group": np.random.choice(list("xyz"), 20),
    "abool": np.random.choice([True, False], 20),
})

# 1. Spark Dataframe Basics

In [5]:
# Convert the pandas dataframe to a spark dataframe.
df = spark.createDataFrame(pandas_dataframe)

In [6]:
# Show the first 3 rows of the dataframe
df.show(3)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
+--------------------+-----+-----+
only showing top 3 rows



In [7]:
# Show the first 7 rows of the dataframe
df.show(7)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
+--------------------+-----+-----+
only showing top 7 rows



In [8]:
# View a summary of the data using `.describe`
df.describe().show()

+-------+-------------------+-----+
|summary|                  n|group|
+-------+-------------------+-----+
|  count|                 20|   20|
|   mean|0.36640264498852165| null|
| stddev| 0.8905322898155364| null|
|    min| -1.261605945319069|    x|
|    max| 2.1503829673811126|    z|
+-------+-------------------+-----+



In [9]:
# Use `.select` to create a new dataframe with just the `n` and `abool` columns. 
# View the first 5 rows of this dataframe.
df.select('n', 'abool').show(5)

+--------------------+-----+
|                   n|abool|
+--------------------+-----+
|  -0.712390662050588|false|
|   0.753766378659703|false|
|-0.04450307833805...|false|
| 0.45181233874578974|false|
|  1.3451017084510097|false|
+--------------------+-----+
only showing top 5 rows



In [10]:
# Use `.select` to create a new dataframe with just the `group` and `abool` columns
# View the first 5 rows of this dataframe.
df.select('group', 'abool').show(5)

+-----+-----+
|group|abool|
+-----+-----+
|    z|false|
|    x|false|
|    z|false|
|    y|false|
|    z|false|
+-----+-----+
only showing top 5 rows



In [11]:
# Use `.select` to create a new dataframe with the `group` column and the `abool` column renamed to `a_boolean_value`
# Show the first 3 rows of this dataframe.
df.select('group', df.abool.alias('a_boolean_value')).show(3)

+-----+---------------+
|group|a_boolean_value|
+-----+---------------+
|    z|          false|
|    x|          false|
|    z|          false|
+-----+---------------+
only showing top 3 rows



In [12]:
# Use `.select` to create a new dataframe with the `group` column and the `n` column renamed to `a_numeric_value`
# Show the first 6 rows of this dataframe

from pyspark.sql.functions import col

df.select('group', col('n').alias('a_numeric_value')).show(6)

+-----+--------------------+
|group|     a_numeric_value|
+-----+--------------------+
|    z|  -0.712390662050588|
|    x|   0.753766378659703|
|    z|-0.04450307833805...|
|    y| 0.45181233874578974|
|    z|  1.3451017084510097|
|    y|  0.5323378882945463|
+-----+--------------------+
only showing top 6 rows



# 2. Column Manipulation

In [13]:
# Use the starter code above to re-create a spark dataframe. Store the spark dataframe in a varaible named `df`
# Use `.select` to add 4 to the `n` column. Show the results.
df.select('n', df.n + 4).show()

+--------------------+------------------+
|                   n|           (n + 4)|
+--------------------+------------------+
|  -0.712390662050588|3.2876093379494122|
|   0.753766378659703| 4.753766378659703|
|-0.04450307833805...|3.9554969216619464|
| 0.45181233874578974|  4.45181233874579|
|  1.3451017084510097|5.3451017084510095|
|  0.5323378882945463| 4.532337888294546|
|  1.3501878997225267| 5.350187899722527|
|  0.8612113741693206|  4.86121137416932|
|  1.4786857374358966| 5.478685737435897|
| -1.0453771305385342| 2.954622869461466|
| -0.7889890249515489|3.2110109750484512|
|  -1.261605945319069| 2.738394054680931|
|  0.5628467852810314| 4.562846785281032|
|-0.24332625188556253|3.7566737481144377|
|  0.9137407048596775| 4.913740704859677|
| 0.31735092273633597| 4.317350922736336|
| 0.12730328020698067| 4.127303280206981|
|  2.1503829673811126| 6.150382967381113|
|  0.6062886568962988| 4.606288656896298|
|-0.02677164998644...|3.9732283500135592|
+--------------------+------------

In [14]:
# Subtract 5 from the `n` column and view the results.
df.select('n', df.n - 5).show()

+--------------------+-------------------+
|                   n|            (n - 5)|
+--------------------+-------------------+
|  -0.712390662050588| -5.712390662050588|
|   0.753766378659703| -4.246233621340297|
|-0.04450307833805...| -5.044503078338053|
| 0.45181233874578974|  -4.54818766125421|
|  1.3451017084510097|-3.6548982915489905|
|  0.5323378882945463| -4.467662111705454|
|  1.3501878997225267|-3.6498121002774733|
|  0.8612113741693206|  -4.13878862583068|
|  1.4786857374358966| -3.521314262564103|
| -1.0453771305385342| -6.045377130538534|
| -0.7889890249515489| -5.788989024951549|
|  -1.261605945319069| -6.261605945319069|
|  0.5628467852810314| -4.437153214718968|
|-0.24332625188556253| -5.243326251885563|
|  0.9137407048596775| -4.086259295140323|
| 0.31735092273633597| -4.682649077263664|
| 0.12730328020698067| -4.872696719793019|
|  2.1503829673811126|-2.8496170326188874|
|  0.6062886568962988| -4.393711343103702|
|-0.02677164998644...| -5.026771649986441|
+----------

In [15]:
# Multiply the `n` column by 2. View the results along with the original numbers.
df.select('n', df.n - 5).show()

+--------------------+-------------------+
|                   n|            (n - 5)|
+--------------------+-------------------+
|  -0.712390662050588| -5.712390662050588|
|   0.753766378659703| -4.246233621340297|
|-0.04450307833805...| -5.044503078338053|
| 0.45181233874578974|  -4.54818766125421|
|  1.3451017084510097|-3.6548982915489905|
|  0.5323378882945463| -4.467662111705454|
|  1.3501878997225267|-3.6498121002774733|
|  0.8612113741693206|  -4.13878862583068|
|  1.4786857374358966| -3.521314262564103|
| -1.0453771305385342| -6.045377130538534|
| -0.7889890249515489| -5.788989024951549|
|  -1.261605945319069| -6.261605945319069|
|  0.5628467852810314| -4.437153214718968|
|-0.24332625188556253| -5.243326251885563|
|  0.9137407048596775| -4.086259295140323|
| 0.31735092273633597| -4.682649077263664|
| 0.12730328020698067| -4.872696719793019|
|  2.1503829673811126|-2.8496170326188874|
|  0.6062886568962988| -4.393711343103702|
|-0.02677164998644...| -5.026771649986441|
+----------

In [17]:
# Add a new column named `n2` that is the `n` value multiplied by -1. Show the first 4 rows of your dataframe. 
# You should see the original `n` value as well as `n2`.
df = df.select('*', (df.n * -1).alias('n2'))
df.show(4)

+--------------------+-----+-----+--------------------+
|                   n|group|abool|                  n2|
+--------------------+-----+-----+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|
|   0.753766378659703|    x|false|  -0.753766378659703|
|-0.04450307833805...|    z|false|0.044503078338053455|
| 0.45181233874578974|    y|false|-0.45181233874578974|
+--------------------+-----+-----+--------------------+
only showing top 4 rows



In [18]:
# Add a new column named `n3` that is the n value squared. Show the first 5 rows of your dataframe. 
# You should see both `n`, `n2`, and `n3`.
df = df.select('*', (df.n * df.n).alias('n3'))
df.show(5)

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
+--------------------+-----+-----+--------------------+--------------------+
only showing top 5 rows



In [19]:
# What happens when you run the code below?
df.group + df.abool
# A Column object is produced that represents the transformation of adding together the group and abool columns.

Column<'(group + abool)'>

In [21]:
# What happens when you run the code below?  
# What is the difference between this and the previous code sample?
df.select(df.group + df.abool)

# An error is produced referencing the incompatible types. 
# Unlike the previous code sample, this one is done within the context of a .select, 
# so even though there are still no values produced (we haven't invoked an action yet), 
# spark is aware that the types are incompatible.

AnalysisException: cannot resolve '(CAST(group AS DOUBLE) + abool)' due to data type mismatch: differing types in '(CAST(group AS DOUBLE) + abool)' (double and boolean).;
'Project [unresolvedalias((cast(group#1 as double) + abool#2), Some(org.apache.spark.sql.Column$$Lambda$3085/0x00000008012fb040@4a5c5eb3))]
+- Project [n#0, group#1, abool#2, n2#253, (n#0 * n#0) AS n3#275]
   +- Project [n#0, group#1, abool#2, (n#0 * cast(-1 as double)) AS n2#253]
      +- LogicalRDD [n#0, group#1, abool#2], false


# 3. Type casting

In [22]:
# Use .printSchema to view the datatypes in your dataframe.
df.printSchema()

root
 |-- n: double (nullable = true)
 |-- group: string (nullable = true)
 |-- abool: boolean (nullable = true)
 |-- n2: double (nullable = true)
 |-- n3: double (nullable = true)



In [24]:
# Use `.dtypes` to view the datatypes in your dataframe.
df.dtypes

[('n', 'double'),
 ('group', 'string'),
 ('abool', 'boolean'),
 ('n2', 'double'),
 ('n3', 'double')]

In [25]:
# What is the difference between the two code samples below?
# One is a creating a Column and one is using that same column in a .select in order to view the results of the cast.
df.abool.cast('int')

Column<'CAST(abool AS INT)'>

In [26]:
df.select(df.abool.cast('int')).show()

+-----+
|abool|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    1|
|    1|
|    0|
|    0|
|    1|
|    1|
|    0|
|    0|
|    0|
|    1|
|    0|
|    1|
+-----+



In [27]:
# Use `.select` and `.cast` to convert the `abool` column to an integer type. View the results.
df.select('abool', df.abool.cast('int')).show()

+-----+-----+
|abool|abool|
+-----+-----+
|false|    0|
|false|    0|
|false|    0|
|false|    0|
|false|    0|
|false|    0|
|false|    0|
|false|    0|
| true|    1|
| true|    1|
|false|    0|
|false|    0|
| true|    1|
| true|    1|
|false|    0|
|false|    0|
|false|    0|
| true|    1|
|false|    0|
| true|    1|
+-----+-----+



In [28]:
#Convert the `group` column to a integer data type and view the results. What happens?
# the values are converted to nulls
df.select('group', df.group.cast('int')).show()

+-----+-----+
|group|group|
+-----+-----+
|    z| null|
|    x| null|
|    z| null|
|    y| null|
|    z| null|
|    y| null|
|    z| null|
|    x| null|
|    z| null|
|    y| null|
|    x| null|
|    y| null|
|    y| null|
|    y| null|
|    y| null|
|    x| null|
|    z| null|
|    y| null|
|    x| null|
|    x| null|
+-----+-----+



In [29]:
# Convert the `n` column to a integer data type and view the results. What happens?
df.select('n', df.n.cast('int')).show()

+--------------------+---+
|                   n|  n|
+--------------------+---+
|  -0.712390662050588|  0|
|   0.753766378659703|  0|
|-0.04450307833805...|  0|
| 0.45181233874578974|  0|
|  1.3451017084510097|  1|
|  0.5323378882945463|  0|
|  1.3501878997225267|  1|
|  0.8612113741693206|  0|
|  1.4786857374358966|  1|
| -1.0453771305385342| -1|
| -0.7889890249515489|  0|
|  -1.261605945319069| -1|
|  0.5628467852810314|  0|
|-0.24332625188556253|  0|
|  0.9137407048596775|  0|
| 0.31735092273633597|  0|
| 0.12730328020698067|  0|
|  2.1503829673811126|  2|
|  0.6062886568962988|  0|
|-0.02677164998644...|  0|
+--------------------+---+



In [30]:
# Convert the `abool` column to a string data type and view the results. What happens?
df.select('abool', df.abool.cast('string')).show()

+-----+-----+
|abool|abool|
+-----+-----+
|false|false|
|false|false|
|false|false|
|false|false|
|false|false|
|false|false|
|false|false|
|false|false|
| true| true|
| true| true|
|false|false|
|false|false|
| true| true|
| true| true|
|false|false|
|false|false|
|false|false|
| true| true|
|false|false|
| true| true|
+-----+-----+



# 4. Built-in Functions

In [31]:
from pyspark.sql.functions import min, max, mean, lit, concat

In [35]:
# Find the highest `n` value
df.select(max('n')).show()

+------------------+
|            max(n)|
+------------------+
|2.1503829673811126|
+------------------+



In [36]:
# Find the lowest `n` value
df.select(min('n')).show()

+------------------+
|            min(n)|
+------------------+
|-1.261605945319069|
+------------------+



In [37]:
# Find the average `n` value
df.select(mean('n')).show()

+-------------------+
|             avg(n)|
+-------------------+
|0.36640264498852165|
+-------------------+



In [44]:
# Use `concat` to change the `group` column to say, e.g. "Group: x" or "Group: y"
df.select(concat(lit('Group: '), 'group')).show()

+----------------------+
|concat(Group: , group)|
+----------------------+
|              Group: z|
|              Group: x|
|              Group: z|
|              Group: y|
|              Group: z|
|              Group: y|
|              Group: z|
|              Group: x|
|              Group: z|
|              Group: y|
|              Group: x|
|              Group: y|
|              Group: y|
|              Group: y|
|              Group: y|
|              Group: x|
|              Group: z|
|              Group: y|
|              Group: x|
|              Group: x|
+----------------------+



In [45]:
df.show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
|  0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|
|  1.3501878997225267|    z|false| -1.3501878997225267|  1.8230073645571279|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
|  1.4786857374358966|    z| true| -1.4786857374358966|  2.1865115100963415|
| -1.0453771305385342|    y| true|  1.0453771305385342|  1.0928133450529796|

In [42]:
# Use `concat` to combine the `n` and `group` columns to produce results that look like this: "x: -1.432" or "z: 2.352"
df.select(concat('group', lit(': '), 'n')).show()

+--------------------+
|concat(group, : , n)|
+--------------------+
|z: -0.71239066205...|
|x: 0.753766378659703|
|z: -0.04450307833...|
|y: 0.451812338745...|
|z: 1.345101708451...|
|y: 0.532337888294...|
|z: 1.350187899722...|
|x: 0.861211374169...|
|z: 1.478685737435...|
|y: -1.04537713053...|
|x: -0.78898902495...|
|y: -1.26160594531...|
|y: 0.562846785281...|
|y: -0.24332625188...|
|y: 0.913740704859...|
|x: 0.317350922736...|
|z: 0.127303280206...|
|y: 2.150382967381...|
|x: 0.606288656896...|
|x: -0.02677164998...|
+--------------------+



# When / Otherwise

In [47]:
from pyspark.sql.functions import when

In [48]:
# Use `when` and `.otherwise` to create a column that contains the text 
# "It is true" when `abool` is true and "It is false"" when `abool` is false.
df.select(when(df.abool, 'It is true').otherwise('It is false')).show()

+----------------------------------------------------+
|CASE WHEN abool THEN It is true ELSE It is false END|
+----------------------------------------------------+
|                                         It is false|
|                                         It is false|
|                                         It is false|
|                                         It is false|
|                                         It is false|
|                                         It is false|
|                                         It is false|
|                                         It is false|
|                                          It is true|
|                                          It is true|
|                                         It is false|
|                                         It is false|
|                                          It is true|
|                                          It is true|
|                                         It is false|
|         

In [49]:
# Create a column that contains 0 if n is less than 0, otherwise, the original n value.
df.select('n', when(df.n < 0, 0).otherwise(df.n)).show()

+--------------------+-----------------------------------+
|                   n|CASE WHEN (n < 0) THEN 0 ELSE n END|
+--------------------+-----------------------------------+
|  -0.712390662050588|                                0.0|
|   0.753766378659703|                  0.753766378659703|
|-0.04450307833805...|                                0.0|
| 0.45181233874578974|                0.45181233874578974|
|  1.3451017084510097|                 1.3451017084510097|
|  0.5323378882945463|                 0.5323378882945463|
|  1.3501878997225267|                 1.3501878997225267|
|  0.8612113741693206|                 0.8612113741693206|
|  1.4786857374358966|                 1.4786857374358966|
| -1.0453771305385342|                                0.0|
| -0.7889890249515489|                                0.0|
|  -1.261605945319069|                                0.0|
|  0.5628467852810314|                 0.5628467852810314|
|-0.24332625188556253|                                0.

# Filter / Where

In [50]:
# Use .filter or .where to select just the rows where the group is y and view the results.
df.filter(df.group == 'y').show()

+--------------------+-----+-----+--------------------+-------------------+
|                   n|group|abool|                  n2|                 n3|
+--------------------+-----+-----+--------------------+-------------------+
| 0.45181233874578974|    y|false|-0.45181233874578974|0.20413438944294027|
|  0.5323378882945463|    y|false| -0.5323378882945463| 0.2833836273138969|
| -1.0453771305385342|    y| true|  1.0453771305385342| 1.0928133450529796|
|  -1.261605945319069|    y|false|   1.261605945319069|  1.591649561264422|
|  0.5628467852810314|    y| true| -0.5628467852810314|0.31679650370119145|
|-0.24332625188556253|    y| true| 0.24332625188556253|0.05920766485667622|
|  0.9137407048596775|    y|false| -0.9137407048596775| 0.8349220757174602|
|  2.1503829673811126|    y| true| -2.1503829673811126|  4.624146906402799|
+--------------------+-----+-----+--------------------+-------------------+



In [51]:
# Select just the columns where the abool column is false and view the results.
df.filter(~ df.abool).show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
|  0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|
|  1.3501878997225267|    z|false| -1.3501878997225267|  1.8230073645571279|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|
|  -1.261605945319069|    y|false|   1.261605945319069|   1.591649561264422|

In [52]:
# Find the columns where the group column is not y.
df.filter(df.group != 'y').show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
|  1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
|  1.3501878997225267|    z|false| -1.3501878997225267|  1.8230073645571279|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
|  1.4786857374358966|    z| true| -1.4786857374358966|  2.1865115100963415|
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|
| 0.31735092273633597|    x|false|-0.31735092273633597| 0.10071160816160388|
| 0.12730328020698067|    z|false|-0.12730328020698067|0.016206125151457036|

In [53]:
# Find the columns where n is positive.
df.filter(df.n > 0).show()

+-------------------+-----+-----+--------------------+--------------------+
|                  n|group|abool|                  n2|                  n3|
+-------------------+-----+-----+--------------------+--------------------+
|  0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
| 1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
| 0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|
| 1.3501878997225267|    z|false| -1.3501878997225267|  1.8230073645571279|
| 0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
| 1.4786857374358966|    z| true| -1.4786857374358966|  2.1865115100963415|
| 0.5628467852810314|    y| true| -0.5628467852810314| 0.31679650370119145|
| 0.9137407048596775|    y|false| -0.9137407048596775|  0.8349220757174602|
|0.31735092273633597|    x|false|-0.31735092273633597| 0.10071160816160388|
|0.127303280

In [54]:
# Find the columns where abool is true and the group column is z.
df.filter(df.abool & (df.group == 'z')).show()

+------------------+-----+-----+-------------------+------------------+
|                 n|group|abool|                 n2|                n3|
+------------------+-----+-----+-------------------+------------------+
|1.4786857374358966|    z| true|-1.4786857374358966|2.1865115100963415|
+------------------+-----+-----+-------------------+------------------+



In [55]:
# Find the columns where abool is true or the group column is z.
df.filter(df.abool | (df.group == 'z')).show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
|  1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
|  1.3501878997225267|    z|false| -1.3501878997225267|  1.8230073645571279|
|  1.4786857374358966|    z| true| -1.4786857374358966|  2.1865115100963415|
| -1.0453771305385342|    y| true|  1.0453771305385342|  1.0928133450529796|
|  0.5628467852810314|    y| true| -0.5628467852810314| 0.31679650370119145|
|-0.24332625188556253|    y| true| 0.24332625188556253| 0.05920766485667622|
| 0.12730328020698067|    z|false|-0.12730328020698067|0.016206125151457036|
|  2.1503829673811126|    y| true| -2.1503829673811126|   4.624146906402799|

In [56]:
# Find the columns where abool is false and n is less than 1
df.filter(~ df.abool & (df.n < 1)).show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|
|  -1.261605945319069|    y|false|   1.261605945319069|   1.591649561264422|
|  0.9137407048596775|    y|false| -0.9137407048596775|  0.8349220757174602|
| 0.31735092273633597|    x|false|-0.31735092273633597| 0.10071160816160388|

In [57]:
# Find the columns where abool is false or n is less than 1
df.filter(~ df.abool | (df.n < 1)).show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
|  0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|
|  1.3501878997225267|    z|false| -1.3501878997225267|  1.8230073645571279|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
| -1.0453771305385342|    y| true|  1.0453771305385342|  1.0928133450529796|
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|

# Sorting

In [58]:
# imports
from pyspark.sql.functions import asc, desc

In [60]:
# Sort by the n value.
df.sort('n').show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|  -1.261605945319069|    y|false|   1.261605945319069|   1.591649561264422|
| -1.0453771305385342|    y| true|  1.0453771305385342|  1.0928133450529796|
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|-0.24332625188556253|    y| true| 0.24332625188556253| 0.05920766485667622|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
|-0.02677164998644...|    x| true|0.026771649986440726|7.167212429964917E-4|
| 0.12730328020698067|    z|false|-0.12730328020698067|0.016206125151457036|
| 0.31735092273633597|    x|false|-0.31735092273633597| 0.10071160816160388|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|

In [65]:
# Sort by the group value, both ascending and descending.
df.sort(asc('group')).show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
|-0.02677164998644...|    x| true|0.026771649986440726|7.167212429964917E-4|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|
| 0.31735092273633597|    x|false|-0.31735092273633597| 0.10071160816160388|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
|  0.6062886568962988|    x|false| -0.6062886568962988|   0.367585935481118|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  0.9137407048596775|    y|false| -0.9137407048596775|  0.8349220757174602|
|-0.24332625188556253|    y| true| 0.24332625188556253| 0.05920766485667622|
|  0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|

In [63]:
# descending
df.sort(desc('group')).show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
| 0.12730328020698067|    z|false|-0.12730328020698067|0.016206125151457036|
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|
|  1.3451017084510097|    z|false| -1.3451017084510097|  1.8092986060778251|
|  1.3501878997225267|    z|false| -1.3501878997225267|  1.8230073645571279|
|-0.04450307833805...|    z|false|0.044503078338053455|0.001980523981562...|
|  1.4786857374358966|    z| true| -1.4786857374358966|  2.1865115100963415|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  0.9137407048596775|    y|false| -0.9137407048596775|  0.8349220757174602|
|-0.24332625188556253|    y| true| 0.24332625188556253| 0.05920766485667622|
|  0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|

In [66]:
# Sort by the group value first, then, within each group, sort by n value.
df.sort('group', 'n').show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|
|-0.02677164998644...|    x| true|0.026771649986440726|7.167212429964917E-4|
| 0.31735092273633597|    x|false|-0.31735092273633597| 0.10071160816160388|
|  0.6062886568962988|    x|false| -0.6062886568962988|   0.367585935481118|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
|  -1.261605945319069|    y|false|   1.261605945319069|   1.591649561264422|
| -1.0453771305385342|    y| true|  1.0453771305385342|  1.0928133450529796|
|-0.24332625188556253|    y| true| 0.24332625188556253| 0.05920766485667622|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|

In [67]:
# Sort by abool, group, and n. Does it matter in what order you specify the columns when sorting?
df.sort('abool', 'group', 'n').show()

+--------------------+-----+-----+--------------------+--------------------+
|                   n|group|abool|                  n2|                  n3|
+--------------------+-----+-----+--------------------+--------------------+
| -0.7889890249515489|    x|false|  0.7889890249515489|  0.6225036814939958|
| 0.31735092273633597|    x|false|-0.31735092273633597| 0.10071160816160388|
|  0.6062886568962988|    x|false| -0.6062886568962988|   0.367585935481118|
|   0.753766378659703|    x|false|  -0.753766378659703|  0.5681637535977627|
|  0.8612113741693206|    x|false| -0.8612113741693206|  0.7416850309986095|
|  -1.261605945319069|    y|false|   1.261605945319069|   1.591649561264422|
| 0.45181233874578974|    y|false|-0.45181233874578974| 0.20413438944294027|
|  0.5323378882945463|    y|false| -0.5323378882945463|  0.2833836273138969|
|  0.9137407048596775|    y|false| -0.9137407048596775|  0.8349220757174602|
|  -0.712390662050588|    z|false|   0.712390662050588|   0.507500455376875|